<a href="https://colab.research.google.com/github/shaffie4488/trident4488/blob/main/LEX_and_YACC_Compiler4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!sudo apt-get install -y flex bison build-essential
!sudo apt-get update


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,006 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64

In [18]:
%%writefile ex4.l
%{
#include<stdio.h>
#include<string.h>
#include "ex4.tab.h"
%}
%%
[ \n\t]+           ;
"main"             { return MAIN; }
"int"|"float"|"double"|"char" {
    strcpy(yylval.string, yytext);
    return TYPE;
}
[a-zA-Z][a-zA-Z0-9_]* {
    strcpy(yylval.string, yytext);
    return ID;
}
[0-9]+\.[0-9]+ {
    yylval.dval = atof(yytext);
    return NUMBER;
}
[0-9]+ {
    yylval.dval = atof(yytext);
    return NUMBER;
}
.                   { return yytext[0]; }
%%
int yywrap() {
    return 1;
}


Writing ex4.l


In [19]:
%%writefile ex4.y
%{
#include<stdio.h>
#include<string.h>
#include<stdlib.h>

int yylex(void);
int yyerror(char *s);

struct Symbol_Table {
    char sym_name[10];
    char sym_type[10];
    double value;
} Sym[10];

int sym_cnt = 0;
int Index = 0;
int temp_var = 0;

int search_symbol(char []);
void make_symtab_entry(char [], char [], double);
void display_sym_tab();
void addQuadruple(char [], char [], char [], char []);
void display_Quadruple();
void push(char *);
char* pop();

struct Quadruple {
    char operator[5];
    char operand1[10];
    char operand2[10];
    char result[10];
} QUAD[25];

struct Stack {
    char *items[10];
    int top;
} Stk;

%}

%union {
    int ival;
    double dval;
    char string[10];
}

%token <dval> NUMBER
%token <string> TYPE
%token <string> ID
%token MAIN
%type <string> varlist expr
%left '+' '-'
%left '*' '/'

%%

program:
    MAIN '(' ')' '{' body '}'
;

body:
    varstmt stmtlist
;

varstmt:
    vardecl varstmt
|   /* empty */
;

vardecl:
    TYPE varlist ';'
;

varlist:
    varlist ',' ID {
        int i = search_symbol($3);
        if(i != -1)
            printf("\nMultiple Declaration of Variable %s", $3);
        else
            make_symtab_entry($3, $<string>1, 0);
    }
|   ID '=' NUMBER {
        int i = search_symbol($1);
        if(i != -1)
            printf("\nMultiple Declaration of Variable %s", $1);
        else
            make_symtab_entry($1, $<string>1, $3);
    }
|   varlist ',' ID '=' NUMBER {
        int i = search_symbol($3);
        if(i != -1)
            printf("\nMultiple Declaration of Variable %s", $3);
        else
            make_symtab_entry($3, $<string>1, $5);
    }
|   ID {
        int i = search_symbol($1);
        if(i != -1)
            printf("\nMultiple Declaration of Variable %s", $1);
        else
            make_symtab_entry($1, $<string>1, 0);
    }
;

stmtlist:
    stmt stmtlist
|   /* empty */
;

stmt:
    ID '=' NUMBER ';' {
        int i = search_symbol($1);
        if(i == -1)
            printf("\nUndefined Variable %s", $1);
        else {
            char temp[10];
            if(strcmp(Sym[i].sym_type, "int") == 0)
                sprintf(temp, "%d", (int)$3);
            else
                snprintf(temp, 10, "%f", $3);
            addQuadruple("=", "", temp, $1);
        }
    }
|   ID '=' ID ';' {
        int i = search_symbol($1);
        int j = search_symbol($3);
        if(i == -1 || j == -1)
            printf("\nUndefined Variable %s or %s", $1, $3);
        else
            addQuadruple("=", "", $3, $1);
    }
|   ID '=' expr ';' {
        addQuadruple("=", "", pop(), $1);
    }
;

expr:
    expr '+' expr {
        char str[5], str1[10] = "t";
        sprintf(str, "%d", temp_var++);
        strcat(str1, str);
        addQuadruple("+", pop(), pop(), str1);
        push(str1);
    }
|   expr '-' expr {
        char str[5], str1[10] = "t";
        sprintf(str, "%d", temp_var++);
        strcat(str1, str);
        addQuadruple("-", pop(), pop(), str1);
        push(str1);
    }
|   expr '*' expr {
        char str[5], str1[10] = "t";
        sprintf(str, "%d", temp_var++);
        strcat(str1, str);
        addQuadruple("*", pop(), pop(), str1);
        push(str1);
    }
|   expr '/' expr {
        char str[5], str1[10] = "t";
        sprintf(str, "%d", temp_var++);
        strcat(str1, str);
        addQuadruple("/", pop(), pop(), str1);
        push(str1);
    }
|   ID {
        int i = search_symbol($1);
        if(i == -1)
            printf("\nUndefined Variable %s", $1);
        else
            push($1);
    }
|   NUMBER {
        char temp[10];
        snprintf(temp, 10, "%f", $1);
        push(temp);
    }
;

%%

extern FILE *yyin;

int main() {
    Stk.top = -1;
    yyin = fopen("input.txt", "r");
    if (!yyin) {
        perror("input.txt");
        return 1;
    }
    yyparse();
    display_sym_tab();
    printf("\n\n");
    display_Quadruple();
    printf("\n\n");
    return 0;
}

int search_symbol(char sym[10]) {
    for(int i = 0; i < sym_cnt; i++) {
        if(strcmp(Sym[i].sym_name, sym) == 0)
            return i;
    }
    return -1;
}

void make_symtab_entry(char sym[10], char dtype[10], double val) {
    strcpy(Sym[sym_cnt].sym_name, sym);
    strcpy(Sym[sym_cnt].sym_type, dtype);
    Sym[sym_cnt].value = val;
    sym_cnt++;
}

void display_sym_tab() {
    printf("\n\nThe Symbol Table\n\n");
    printf("Name\tType\tValue\n");
    for(int i = 0; i < sym_cnt; i++)
        printf("%s\t%s\t%f\n", Sym[i].sym_name, Sym[i].sym_type, Sym[i].value);
}

void display_Quadruple() {
    printf("\n\nThe INTERMEDIATE CODE Is:\n\n");
    printf("Index\tResult\tOperator\tOperand1\tOperand2\n");
    for(int i = 0; i < Index; i++)
        printf("%d\t%s\t%s\t\t%s\t\t%s\n", i, QUAD[i].result, QUAD[i].operator, QUAD[i].operand1, QUAD[i].operand2);
}

int yyerror(char *s) {
    printf("\nERROR!! %s\n", s);
    return 1;
}

void push(char *str) {
    if (Stk.top >= 9) {
        printf("\nStack Overflow!!\n");
        exit(1);
    }
    Stk.top++;
    Stk.items[Stk.top] = (char *)malloc(strlen(str) + 1);
    strcpy(Stk.items[Stk.top], str);
}

char* pop() {
    if (Stk.top == -1) {
        printf("\nStack Empty!!\n");
        exit(1);
    }
    char *str = (char *)malloc(strlen(Stk.items[Stk.top]) + 1);
    strcpy(str, Stk.items[Stk.top]);
    free(Stk.items[Stk.top]);
    Stk.top--;
    return str;
}

void addQuadruple(char op[10], char op2[10], char op1[10], char res[10]) {
    strcpy(QUAD[Index].operator, op);
    strcpy(QUAD[Index].operand2, op2);
    strcpy(QUAD[Index].operand1, op1);
    strcpy(QUAD[Index].result, res);
    Index++;
}


Writing ex4.y


In [20]:
%%writefile input.txt
main() {
    int a, b;
    a = 5;
    b = a + 3;
}


Writing input.txt


In [21]:
!bison -d ex4.y
!flex ex4.l
!gcc ex4.tab.c lex.yy.c -o parser -lfl


ex4.y: warning: 2 shift/reduce conflicts [-Wconflicts-sr]
ex4.y: note: rerun with option '-Wcounterexamples' to generate conflict counterexamples


In [22]:
!./parser




The Symbol Table

Name	Type	Value
a	a	0.000000
b	a	0.000000




The INTERMEDIATE CODE Is:

Index	Result	Operator	Operand1	Operand2
0	a	=		5.000000		
1	t0	+		3.000000		a
2	b	=		t0		


